In [10]:
#%%
##JOSE MARIA GOMEZ PULIDO
##ALVARO VELASCO DIAZ
##EVA SANCHEZ MUÑOZ
import numpy as np         # arrays    
import sounddevice as sd   # modulo de conexión con portAudio
import soundfile as sf     # para lectura/escritura de wavs
import time
CHUNK = 512
SRATE = 48000

In [18]:
from kbhit import KBHit
class Osc:
    def __init__(self, freq = 440, volume = 1, phase = 0):
        self.freq = freq
        self.volume = volume
        self.phase = phase
        self.frame = -0


    def next(self):
        bloque = np.float32(np.arange(self.frame, self.frame+CHUNK))
        self.frame+=CHUNK
        bloque = np.sin(self.phase + bloque*2*np.pi*self.freq/SRATE)
        bloque*=self.volume
        return bloque

    def getVolume(self):
        return self.volume
    def setVolume(self, volume):
        self.volume = volume
        if self.volume < 0:
            self.volume = 0
        elif self.volume > 1:
            self.volume = 1
        
    def getFrequency(self):
        return self.freq
    def setFrequency(self, freq):
        self.freq = freq

    def noteOn(self, note):
        frecuencias = {
            "C": 523.251,
            "D": 587.33,
            "E": 659.255,
            "F": 698.456,
            "G": 783.991,
            "A": 880,
            "B": 987.787
        }
        
        self.freq = frecuencias[note.upper()]
        if note != note.upper():
            self.freq *= 2

    def noteOff(self):
        self.freq = 0


In [20]:
BPM = 120


partitura = [
    ("G", 1/2),
    ("G", 1/2),
    ("A", 1),
    ("G", 1),
    ("c", 1),
    ("B", 2)
]


osc = Osc()
input = KBHit()
end = False
stream = sd.OutputStream(samplerate=SRATE, blocksize=CHUNK, channels=1)
deltaTime = CHUNK/SRATE
stream.start()
for nota in partitura:
    duration = nota[1]/BPM*60
    osc.noteOn(nota[0])
    while duration > 0:
        bloque = osc.next()
        duration-=deltaTime
        stream.write(bloque)
    duration = BPM/60.0 * 0.010
    osc.noteOff()
    while duration > 0:
        bloque = osc.next()
        duration-=deltaTime
        stream.write(bloque)

    
input.quit()
stream.stop()
stream.close()